# DeckGL Backend Example

This notebook demonstrates how to use the DeckGL backend in anymap, which builds upon the MapLibre backend to provide advanced data visualization capabilities using DeckGL layers.

## Installation

Make sure you have anymap installed:

```bash
pip install anymap
```

## Troubleshooting

If you encounter widget loading issues:

1. **Restart the kernel** and clear all outputs
2. **Check internet connectivity** - DeckGL loads from CDN
3. **Open browser console** (F12) to see any JavaScript errors
4. **Try a simple example first** before complex visualizations

The widget dynamically loads DeckGL and MapLibre from CDN, so it requires an internet connection.

## Basic Setup

First, let's import the necessary modules and create a basic DeckGL map:

In [ ]:
from anymap import DeckGLMap
import json
import requests

## Simple Test First

Let's start with a very simple DeckGL map to ensure the widget loads correctly:

In [ ]:
# Create a simple test map first
simple_map = DeckGLMap(center=[0, 0], zoom=2, height="600px")
print("✓ Simple DeckGL map created successfully")
print(f"Center: {simple_map.center}, Zoom: {simple_map.zoom}")

# Display the simple map
simple_map

In [ ]:
# Create a basic DeckGL map
m = DeckGLMap(
    center=[51.47, 0.45],  # London
    zoom=4,
    bearing=0,
    pitch=30,
    height="600px",
    map_style="https://basemaps.cartocdn.com/gl/positron-nolabels-gl-style/style.json",
)

print(f"Created DeckGL map with center: {m.center}, zoom: {m.zoom}")
print(f"Map style: {m.map_style}")
print(f"Current DeckGL layers: {len(m.deckgl_layers)}")

In [ ]:
m

## Adding GeoJSON Layer

Let's add a GeoJSON layer showing airports around the world:

In [ ]:
# Add airports GeoJSON layer
airports_url = (
    "https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_10m_airports.geojson"
)

m.add_geojson_layer(
    layer_id="airports",
    geojson_data=airports_url,
    filled=True,
    pointRadiusMinPixels=2,
    pointRadiusScale=2000,
    getPointRadius="@@=11 - f.properties.scalerank",  # DeckGL expression
    getFillColor=[200, 0, 80, 180],
    pickable=True,
    autoHighlight=True,
)

print(f"Added airports layer. Total layers: {len(m.deckgl_layers)}")

## Adding Arc Layer

Now let's add an Arc layer showing connections from London to major airports. This will display curved lines connecting London to airports with scalerank < 4:

In [ ]:
# Add arc layer showing connections from London
m.add_arc_layer(
    layer_id="arcs",
    data=airports_url,
    dataTransform="@@=d => d.features.filter(f => f.properties.scalerank < 4)",  # Filter major airports
    getSourcePosition="@@=f => [-0.4531566, 51.4709959]",  # London coordinates (function)
    getTargetPosition="@@=f => f.geometry.coordinates",  # Target airport coordinates
    getSourceColor=[0, 128, 200],
    getTargetColor=[200, 0, 80],
    getWidth=1,
    pickable=True,
)

print(f"Added arcs layer. Total layers: {len(m.deckgl_layers)}")

## Adding Scatterplot Layer

Let's also add a scatterplot layer with custom styling:

In [ ]:
# Create sample data for scatterplot
sample_data = [
    {"position": [-0.1276, 51.5074], "name": "London", "size": 1000},
    {"position": [2.3522, 48.8566], "name": "Paris", "size": 800},
    {"position": [13.4050, 52.5200], "name": "Berlin", "size": 600},
    {"position": [12.4964, 41.9028], "name": "Rome", "size": 700},
    {"position": [-3.7038, 40.4168], "name": "Madrid", "size": 650},
]

m.add_scatterplot_layer(
    layer_id="cities",
    data=sample_data,
    get_position="@@=d.position",
    get_radius="@@=d.size",
    get_fill_color=[255, 140, 0, 200],
    radiusMinPixels=10,
    radiusMaxPixels=100,
    pickable=True,
)

print(f"Added cities scatterplot layer. Total layers: {len(m.deckgl_layers)}")

## Display the Map

Now let's display the interactive map (note: this will only work in a Jupyter environment with widget support):

In [ ]:
# Display the map
m

## Export to HTML

Let's export the map to a standalone HTML file that can be opened in any browser:

In [ ]:
# Export to HTML
html_content = m.to_html(
    filename="deckgl_example_export.html",
    title="DeckGL Map Example - Airports and Cities",
    width="100%",
    height="600px",
)

print(f"HTML exported successfully!")
print(f"File saved as: deckgl_example_export.html")
print(f"HTML content length: {len(html_content)} characters")

## Map Interaction Methods

The DeckGL map supports various interaction methods inherited from the MapLibre backend:

In [ ]:
# Change map view
print("Current center:", m.center)
print("Current zoom:", m.zoom)

# Update map center and zoom
m.set_center(48.8566, 2.3522)  # Paris
m.set_zoom(6)

print("New center:", m.center)
print("New zoom:", m.zoom)

In [ ]:
# Change bearing and pitch
m.set_bearing(45)
m.set_pitch(60)

print("New bearing:", m.bearing)
print("New pitch:", m.pitch)

## Layer Management

You can manage DeckGL layers dynamically:

In [ ]:
# Show current layers
print("Current DeckGL layers:")
for i, layer in enumerate(m.deckgl_layers):
    print(
        f"  {i+1}. {layer.get('id', 'Unknown')} ({layer.get('@@type', 'Unknown type')})"
    )

# Remove a layer
m.remove_deckgl_layer("cities")
print(f"\nAfter removing cities layer: {len(m.deckgl_layers)} layers remaining")

In [ ]:
# Add the cities layer back with different styling
m.add_scatterplot_layer(
    layer_id="cities_styled",
    data=sample_data,
    get_position="@@=d.position",
    get_radius="@@=d.size * 0.5",
    get_fill_color=[0, 255, 0, 180],  # Green color
    radiusMinPixels=5,
    radiusMaxPixels=50,
    pickable=True,
)

print(f"Added styled cities layer. Total layers: {len(m.deckgl_layers)}")

## Export Final Version

Let's export the final version with all our changes:

In [ ]:
# Export final version
final_html = m.to_html(
    filename="deckgl_final_example.html",
    title="Final DeckGL Map - Airports, Arcs, and Cities",
    width="100%",
    height="800px",
)

print("Final HTML exported successfully!")
print("File saved as: deckgl_final_example.html")
print(f"Final map has {len(m.deckgl_layers)} DeckGL layers")
print(
    f"Final view: center={m.center}, zoom={m.zoom}, bearing={m.bearing}, pitch={m.pitch}"
)

## Summary

This notebook demonstrated:

1. **Basic DeckGL Map Creation**: Creating a DeckGL map with custom styling and view settings
2. **GeoJSON Layer**: Adding interactive airport data with custom styling
3. **Arc Layer**: Visualizing connections between geographic points
4. **Scatterplot Layer**: Displaying point data with custom sizing and colors
5. **Map Interaction**: Changing view, bearing, and pitch programmatically
6. **Layer Management**: Adding and removing layers dynamically
7. **HTML Export**: Exporting interactive maps to standalone HTML files

The DeckGL backend provides powerful visualization capabilities while maintaining compatibility with the anymap framework. The exported HTML files can be shared and viewed in any modern web browser without requiring a Python environment.